In [1]:
print("hello vipyne")

hello vipyne


In [1]:
pip install pipecat-ai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install nvidia-riva-client

  Using cached setuptools-70.0.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached protobuf-5.29.1-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 7.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 11.4 MB/s eta 0:00:00a 0:00:01
Using cached setuptools-70.0.0-py3-none-any.whl (863 kB)
Using cached protobuf-5.29.1-cp38-abi3-macosx_10_9_universal2.whl (417 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install nest_asyncio


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
#
# Copyright (c) 2024, Daily
#
# SPDX-License-Identifier: BSD 2-Clause License
#

import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()
import os
import sys

from loguru import logger

from pipecat.audio.vad.silero import SileroVADAnalyzer
from pipecat.frames.frames import LLMMessagesFrame, EndFrame
from pipecat.pipeline.pipeline import Pipeline
from pipecat.pipeline.runner import PipelineRunner
from pipecat.pipeline.task import PipelineParams, PipelineTask
from pipecat.processors.aggregators.openai_llm_context import OpenAILLMContext
from pipecat.services.nim import NimLLMService
from pipecat.services.riva import FastPitchTTSService, ParakeetSTTService
from pipecat.transports.services.daily import DailyParams, DailyTransport
from pipecat.transports.services.helpers.daily_rest import DailyRESTHelper

async def configure(aiohttp_session: aiohttp.ClientSession):
    url = os.getenv("DAILY_SAMPLE_ROOM_URL")
    key = os.getenv("DAILY_API_KEY")

    if not url:
        raise Exception(
            "No Daily room specified. use the -u/--url option from the command line, or set DAILY_SAMPLE_ROOM_URL in your environment to specify a Daily room URL."
        )

    if not key:
        raise Exception(
            "No Daily API key specified. use the -k/--apikey option from the command line, or set DAILY_API_KEY in your environment to specify a Daily API key, available from https://dashboard.daily.co/developers."
        )

    daily_rest_helper = DailyRESTHelper(
        daily_api_key=key,
        daily_api_url=os.getenv("DAILY_API_URL", "https://api.daily.co/v1"),
        aiohttp_session=aiohttp_session,
    )

    # Create a meeting token for the given room with an expiration 1 hour in
    # the future.
    expiry_time: float = 60 * 60

    token = await daily_rest_helper.get_token(url, expiry_time)

    return (url, token)
    return (url, token)

logger.add(sys.stderr, level="DEBUG")

async def main():
    async with aiohttp.ClientSession() as session:
        (room_url, _) = await configure(session)

        transport = DailyTransport(
            room_url,
            None,
            "Respond bot",
            DailyParams(
                audio_out_enabled=True,
                vad_enabled=True,
                vad_analyzer=SileroVADAnalyzer(),
                vad_audio_passthrough=True,
            ),
        )

        stt = ParakeetSTTService(api_key=os.getenv("NVIDIA_API_KEY"))

        llm = NimLLMService(
            api_key=os.getenv("NVIDIA_API_KEY")
            # api_key=os.getenv("NVIDIA_API_KEY"), model="meta/llama-3.1-405b-instruct"
        )

        tts = FastPitchTTSService(api_key=os.getenv("NVIDIA_API_KEY"))

        messages = [
            {
                "role": "system",
                "content": "You are a helpful LLM in a WebRTC call. Your goal is to demonstrate your capabilities in a succinct way. Your output will be converted to audio so don't include special characters in your answers. Respond to what the user said in a creative and helpful way.",
            },
        ]

        context = OpenAILLMContext(messages)
        context_aggregator = llm.create_context_aggregator(context)

        pipeline = Pipeline(
            [
                transport.input(),  # Transport user input
                stt,  # STT
                context_aggregator.user(),  # User responses
                llm,  # LLM
                tts,  # TTS
                transport.output(),  # Transport bot output
                context_aggregator.assistant(),  # Assistant spoken responses
            ]
        )

        task = PipelineTask(pipeline, PipelineParams(allow_interruptions=True))

        @transport.event_handler("on_first_participant_joined")
        async def on_first_participant_joined(transport, participant):
            # Kick off the conversation.
            messages.append({"role": "system", "content": "Please introduce yourself to the user."})
            await task.queue_frames([LLMMessagesFrame(messages)])

        @transport.event_handler("on_participant_left")
        async def on_participant_left(transport, participant, reason):
            print(f"Participant left: {participant}")
            await task.queue_frame(EndFrame())            

        runner = PipelineRunner()

        await runner.run(task)


if __name__ == "__main__":
    asyncio.run(main())


2024-12-12 13:31:13.281 | INFO     | pipecat.audio.vad.vad_analyzer:set_params:69 - Setting VAD params to: confidence=0.7 start_secs=0.2 stop_secs=0.8 min_volume=0.6
2024-12-12 13:31:13.281 | INFO     | pipecat.audio.vad.vad_analyzer:set_params:69 - Setting VAD params to: confidence=0.7 start_secs=0.2 stop_secs=0.8 min_volume=0.6
2024-12-12 13:31:13.283 | DEBUG    | pipecat.audio.vad.silero:__init__:114 - Loading Silero VAD model...
2024-12-12 13:31:13.283 | DEBUG    | pipecat.audio.vad.silero:__init__:114 - Loading Silero VAD model...
2024-12-12 13:31:13.417 | DEBUG    | pipecat.audio.vad.silero:__init__:136 - Loaded Silero VAD
2024-12-12 13:31:13.417 | DEBUG    | pipecat.audio.vad.silero:__init__:136 - Loaded Silero VAD
2024-12-12 13:31:14.775 | DEBUG    | pipecat.processors.frame_processor:link:150 - Linking PipelineSource#0 -> DailyInputTransport#0
2024-12-12 13:31:14.775 | DEBUG    | pipecat.processors.frame_processor:link:150 - Linking PipelineSource#0 -> DailyInputTransport#0
20

Participant left: {'id': '1ba6bb85-ded7-47ed-a3c7-66224c686d16', 'info': {'permissions': {'canSend': ['customAudio', 'screenAudio', 'screenVideo', 'customVideo', 'microphone', 'camera'], 'canAdmin': [], 'hasPresence': True}, 'userName': 'vanessa', 'isLocal': False, 'isOwner': False, 'joinedAt': 1734031720}}


2024-12-12 13:34:43.621 | DEBUG    | pipecat.transports.base_output:_bot_stopped_speaking:218 - Bot stopped speaking
2024-12-12 13:34:43.621 | DEBUG    | pipecat.transports.base_output:_bot_stopped_speaking:218 - Bot stopped speaking
2024-12-12 13:34:43.622 | INFO     | pipecat.transports.services.daily:leave:435 - Leaving https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/prod-test
2024-12-12 13:34:43.622 | INFO     | pipecat.transports.services.daily:leave:435 - Leaving https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/prod-test
2024-12-12 13:34:43.638 | INFO     | pipecat.transports.services.daily:leave:443 - Left https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/prod-test
2024-12-12 13:34:43.638 | INFO     | pipecat.transports.services.daily:leave:443 - Left https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/prod-test
2024-12-12 13:34:43.641 | DEBUG    | pipecat.pipeline.runner:run:31 - Runner PipelineRunner#0 finished running PipelineTask#0
2024-12-12 13:34:43.641 | DEBU